In [1]:
!pip install xlsxwriter
import pandas as pd
import numpy as np
from google.colab import files
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import statistics
import xlsxwriter
import random
import datetime
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.3 MB/s 


In [2]:
## For the program to work as presented here, run on Google Colab

file_name=files.upload()
name='Economic.csv'

Saving Economic.csv to Economic.csv


In [3]:
## Checking that file uploaded correctly 
print(name)

Economic.csv


# Processing the data





In [11]:
## The working hypothesis is that the patterns of deceleration and acceleration of the economy, and hence its four stages, can be tracked through LEI monthly, 
## quarterly and semi-annual changes. 
## In this workbook we estimate the changes in LEI and create a new file with the columns needed for visualizations and further analysis 

## Creating a variable for the name of output file so succesive versions can be created after modificactions by file_name
output_file_name="LEI_deltas"
output_file_description="-" #optional file description 

In [12]:
## Reading the file  
df=pd.read_csv(name, sep=';')
rows=df.shape[0] #number of rows
columns=df.shape[1]  #number of columns
print(rows)
print (columns)
df.head()

600
7


,month,LEI,CEI,GDP,NBER,Fed_Funds,Inflation
0,7/01/1972,50.3,39.5,3.8,expansion,4.55,2.9
1,8/01/1972,51.0,39.9,3.8,expansion,4.81,2.9
2,9/01/1972,51.6,40.2,3.8,expansion,4.87,3.2
3,10/01/1972,52.0,40.7,6.9,expansion,5.05,3.4
4,11/01/1972,52.5,41.0,6.9,expansion,5.06,3.7


The dataset has monthly info dating back to July 1972, the Leading Economic Indicator (LEI), the Coincident Economic Index (CEI), quarterly GDP (applied equally to every month in the quarter), the effective Fed Funds rate and CPI inflation. The column NBER lists the institute's classification of the given month with regards to the economic cycle, as either expansion or contraction. 



In [13]:
## checking if the datatypes correspond
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   month      600 non-null    object 
 1   LEI        600 non-null    float64
 2   CEI        600 non-null    float64
 3   GDP        600 non-null    float64
 4   NBER       600 non-null    object 
 5   Fed_Funds  600 non-null    float64
 6   Inflation  600 non-null    float64
dtypes: float64(5), object(2)
memory usage: 32.9+ KB


In [14]:
## The dates loaded the first day of the month (this is not actually true as LEI comes out the 3rd Thursday). Let's just keep the month marking
## Changing month to datetime format, because it's an object
df['month']=df['month'].astype('datetime64')

In [15]:
## Changing the format to just month/year
df['month'] = df['month'].dt.strftime('%m/%Y') 

##strftime turns it back into a string type, but since the observations are already ordered it won't matter much when producing visualizations 

In [16]:
## Checking that the format changed 
df.head()

,month,LEI,CEI,GDP,NBER,Fed_Funds,Inflation
0,07/1972,50.3,39.5,3.8,expansion,4.55,2.9
1,08/1972,51.0,39.9,3.8,expansion,4.81,2.9
2,09/1972,51.6,40.2,3.8,expansion,4.87,3.2
3,10/1972,52.0,40.7,6.9,expansion,5.05,3.4
4,11/1972,52.5,41.0,6.9,expansion,5.06,3.7


In [18]:

## These are the change variables we want to include in the dataframe for analysis
delta_1m_LEI=0   ## change in one month
delta_3m_LEI=0   ## change in three months
delta_6m_LEI=0   ## change in six months
sign_delta_1m_LEI=0 ## the sing of the change in delta _1m
sign_delta_3m_LEI=0 ## the sing of the change in delta _3m
sign_delta_6m_LEI=0 ## the sing of the change in delta _6m


## Initializing lists to store the values of estimations to later export to a new CSV file. Delta lists start with zero, since no estimations have been performed yet.  
list_month=[df["month"][0]] 
list_CEI=[df["CEI"][0]] 
list_NBER=[df["NBER"][0]] 
list_delta_1m_LEI=[0]
list_delta_3m_LEI=[0]
list_delta_6m_LEI=[0]
list_sign_delta_1m_LEI=[0]
list_sign_delta_3m_LEI=[0]
list_sign_delta_6m_LEI=[0]


In [19]:
### Estimating changes in LEI and the signs and appending to corresponding lists
### For the sign_delta variables, 1 is assigned to positive changes and 0 to negative changes  
for i in range(1,rows,1):
  delta_1m_LEI=((df["LEI"][i]-df["LEI"][i-1])/df["LEI"][i-1])
  if i>=3:
    delta_3m_LEI=((df["LEI"][i]-df["LEI"][i-3])/df["LEI"][i-3])
  if i>=6:
    delta_6m_LEI=((df["LEI"][i]-df["LEI"][i-6])/df["LEI"][i-6])
  if delta_1m_LEI > 0:
    sign_delta_1m_LEI=1
  else: 
    sign_delta_1m_LEI=0      
  if delta_3m_LEI > 0:
    sign_delta_3m_LEI=1
  else: 
    sign_delta_3m_LEI=0 
  if delta_6m_LEI > 0:
    sign_delta_6m_LEI=1
  else: 
    sign_delta_6m_LEI=0 

  list_delta_1m_LEI.append(delta_1m_LEI)
  list_delta_3m_LEI.append(delta_3m_LEI)
  list_delta_6m_LEI.append(delta_6m_LEI)
 
  list_sign_delta_1m_LEI.append(sign_delta_1m_LEI)
  list_sign_delta_3m_LEI.append(sign_delta_3m_LEI)
  list_sign_delta_6m_LEI.append(sign_delta_6m_LEI)

## extracting the correspoding values for new csv that will be created with the delta and sign columns   
  list_month.append(df["month"][i]) 
  list_LEI.append(df["LEI"][i])
  list_CEI.append(df["CEI"][i]) 
  list_NBER.append(df["NBER"][i]) 

In [20]:
### creating new csv file with the calculated columns and others necessary for visualizations from initial dataframe

direc="/content/"
text=output_file_name+'.csv'
data_output={
  
    "month":list_month,
    "LEI": list_LEI,
    "CEI":list_CEI,
    "NBER": list_NBER,
    "delta_1m_LEI":list_delta_1m_LEI,
    "delta_3m_LEI":list_delta_3m_LEI,
    "delta_6m_LEI":list_delta_6m_LEI,
    "sign_delta_1m_LEI": list_sign_delta_1m_LEI,
    "sign_delta_3m_LEI": list_sign_delta_3m_LEI,
    "sign_delta_6m_LEI": list_sign_delta_6m_LEI,
 
    }
 
df2 = pd.DataFrame(data=data_output)
df2.to_csv(text,index=False) 
file_name=direc+text
files.download(file_name)

## The file downloads automatically. I have loaded this file to the Github repository data folder (LEI_analysis.csv) with the following modification: 
## The zeros for the first missing deltas (1m, 3m and 6m) in 1972 are estimated using the forward three month average of the estimations in the same column. 
## This approach is viable since its in the middle of an expansion period and will not taint the objective of the analysis, which is to determine changes in the cycle.  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>